## Физически информированные нейронные сети

- это тип нейронных сетей, разработанный для решения задач, связанных с дифференциальными уравнениями (обыкновенными и в частных производных), а также для моделирования физических процессов.


### **Основная идея PINN**  
Обычные нейронные сети обучаются на данных, но не учитывают фундаментальные физические законы. PINN же включают физические уравнения (например, уравнение теплопроводности или волновое уравнения) прямо в процесс обучения, что позволяет им:  
1. Требовать **меньше данных** (или вообще никаких) для обучения.  
2. Давать **физически осмысленные** предсказания.  
3. Решать **обратные задачи**.  

### **Как работают PINN?**  

![net](./lecture9/pinn_net.png)

1. **Архитектура**:  
   - Обычно это полносвязная нейронная сеть (MLP), которая принимает на вход координаты (например, $x$, $t$) и выдает решение $u(x, t)$.  
   - Например, для уравнения теплопроводности:  
     $$
     \frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}
     $$
     сеть обучается предсказывать $u(x, t)$, удовлетворяющее этому уравнению.  

2. **Функция потерь (Loss function)**:  
   - Состоит из двух частей:  
     - **Data loss** (потери по данным) — сравнение предсказаний с имеющимися экспериментальными данными.  
     - **Physics loss** (физические потери) — штраф за нарушение уравнений.  
   - Например, для уравнения теплопроводности:  
     $$
     \mathcal{L} = \mathcal{L}_{data} + \lambda \mathcal{L}_{physics}
     $$
     где:  
     $$
     \mathcal{L}_{physics} = \left\| \frac{\partial u}{\partial t} - \alpha \frac{\partial^2 u}{\partial x^2} \right\|^2
     $$
     (норма невязки уравнения).  

3. **Автоматическое дифференцирование (Autograd)**:  
   - Для вычисления производных $\frac{\partial u}{\partial t}$ и $\frac{\partial^2 u}{\partial x^2}$ используется автоматическое дифференцирование (через backpropagation), что позволяет точно учитывать физику.  

4. **Обучение**:  
   - Сеть обучается методом градиентного спуска (например, Adam), минимизируя общий loss.  

### **Недостатки**  
❌ **Вычислительная сложность** — расчет производных может быть затратным.  
❌ **Чувствительность к гиперпараметрам** — баланс между $\mathcal{L}_{data}$ и $\mathcal{L}_{physics}$ требует настройки.  

## Использование PINN для решения оптических задач 
- Нелинейное уравнение Шредингера

**Physics-informed Neural Network for Nonlinear Dynamics in Fiber Optics**
Xiaotian Jiang, Danshi Wang, Qirui Fan, Min Zhang, Chao Lu, Alan Pak Tao Lau
physics > arXiv:2109.00526

- Обратные задачи нино-оптики и оптики метаматероалов
  
** Physics-informed neural networks for inverse problems in nano-optics and metamaterials**
Optics Express 202028(8):11618-11633
DOI:10.1364/OE.384875
![meta](./lecture9/pinn_meta.png)

- Много информации по PINN

https://github.com/idrl-lab/PINNpapers

Для примера рассмотрим задачу с 1-м 2-х уровневым атомом в одномодовом резонаторе. В представлении взаимодействи Гамильтониан этой системы выглядит так
$$
V = g\hbar (a^\dagger \sigma_{-}e^{-i\Delta t} + a\sigma_{+}e^{i\Delta t})
$$
где $g$ - константа взаимодействия атома и поля резонатора, $\Delta$ - отстройка частоты резонатора от атомного перехода.

Ограничимся максимум одним фотоном в резонаторе и представим состояние системы в виде
$$
|\psi \rangle = c_{a1} |a, 1\rangle + c_{b0} |b, 0\rangle
$$

Тогда уранение Шредингера сведется к системе
$$
\frac{dc_{a1}}{dt} = -i g c_{b0} \exp (-i\Delta t)
$$
$$
\frac{dc_{b0}}{dt} = -i g c_{a1} \exp (i\Delta t)
$$

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Параметры задачи
gamma = 1.0 # Константа связи
Delta = 1.0 # отстройка резонатор от перехода
epochs = 10000  # Увеличим количество эпох
n_samples = 5000  # Увеличим число точек

# Генерация данных как тензоров TensorFlow
t_initial = tf.constant([[0.0]], dtype=tf.float32)  # 2D тензор для начального условия
t_domain = tf.reshape(tf.linspace(0.0, 2*np.pi, n_samples), (-1, 1))  # Коллокационные точки

In [7]:
# Архитектура сети
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='tanh', input_shape=(1,), kernel_initializer='random_normal'),
    tf.keras.layers.Dense(64, activation='tanh', kernel_initializer='random_normal'),
    tf.keras.layers.Dense(64, activation='tanh', kernel_initializer='random_normal'),
    tf.keras.layers.Dense(4, kernel_initializer='random_normal')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

C:\Users\denis\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,708 (34.02 KB)

 Trainable params: 8,708 (34.02 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
def compute_loss():
    # Начальные условия
    c0_pred = model(t_initial)
    ca1_r0 = 0.0
    ca1_i0 = 0.0
    cb0_r0 = 1.0
    cb0_i0 = 0.0
    ca1_r0_pred = c0_pred[:, 0:1]
    ca1_i0_pred = c0_pred[:, 1:2]
    cb0_r0_pred = c0_pred[:, 2:3]
    cb0_i0_pred = c0_pred[:, 3:4]

    # MSE для начальных условий
    loss_ic = tf.square(ca1_r0_pred - ca1_r0) + tf.square(ca1_i0_pred - ca1_i0) + \
    tf.square(cb0_r0_pred - cb0_r0) + tf.square(cb0_i0_pred - cb0_i0)

    # Часть 2: Уравнение
    t_colloc = tf.convert_to_tensor(t_domain)  # Убедимся, что это тензор
    t_colloc = tf.reshape(t_colloc, (-1, 1))  # Гарантируем правильную форму

    with tf.GradientTape(persistent=True) as gg:
        gg.watch(t_colloc)
        y = model(t_colloc)
        ca1_r = y[:,0:1]
        ca1_i = y[:,1:2]
        cb0_r = y[:,2:3]
        cb0_i = y[:,3:4]

    dca1_r = gg.gradient(ca1_r, t_colloc)  # вычисление производной координаты
    dca1_i = gg.gradient(ca1_i, t_colloc)  # вычисление производной импулься
    dcb0_r = gg.gradient(cb0_r, t_colloc)  # вычисление производной координаты
    dcb0_i = gg.gradient(cb0_i, t_colloc)  # вычисление производной импулься

    equation1 = dca1_r - gamma*(cb0_i*tf.cos(Delta*t_colloc) - cb0_r*tf.sin(Delta*t_colloc))
    equation2 = dca1_i + gamma*(cb0_r*tf.cos(Delta*t_colloc) + cb0_i*tf.sin(Delta*t_colloc))
    equation3 = dcb0_r - gamma*(ca1_i*tf.cos(Delta*t_colloc) + ca1_r*tf.sin(Delta*t_colloc))
    equation4 = dcb0_i + gamma*(ca1_r*tf.cos(Delta*t_colloc) - ca1_i*tf.sin(Delta*t_colloc))

    loss_eq = tf.reduce_mean(tf.square(equation1)) + tf.reduce_mean(tf.square(equation2)) + \
tf.reduce_mean(tf.square(equation3)) + tf.reduce_mean(tf.square(equation4))

    del gg
    return loss_eq + loss_ic

In [13]:
compute_loss().numpy()[0,0]

1.0006698

In [ ]:
# Обучение
loss_history = []
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = compute_loss()
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    loss_history.append(loss.numpy()[0,0])

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()[0,0]:.6f}')


In [ ]:
# Визуализация
import numpy as np
from scipy.integrate import odeint

def f(y, t, g, d):
  ca1_r, ca1_i, cb0_r, cb0_i = y
  dca1_r  = g*(cb0_i*np.cos(d*t) - cb0_r*np.sin(d*t))
  dca1_i = - g*(cb0_r*np.cos(d*t) + cb0_i*np.sin(d*t))
  dcb0_r = g*(ca1_i*np.cos(d*t) + ca1_r*np.sin(d*t))
  dcb0_i = - g*(ca1_r*np.cos(d*t) - ca1_i*np.sin(d*t))
  return [dca1_r, dca1_i, dcb0_r, dcb0_i]

# Initial conditions
y0 = [0.0, 0.0, 1.0, 0.0]

# Time points
t = np.linspace(0, 2*np.pi, 500)

# Solve ODE
sol = odeint(f, y0, t, args=(gamma, Delta))

t_test = t.reshape(-1, 1)
y_pred = model.predict(t_test, verbose=0)[:,0]**2 + model.predict(t_test, verbose=0)[:,1]**2
y_true = np.cos(0.5*Omega_R*t_test)**2

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(t_test, sol[:, 0]**2 + sol[:, 1]**2, label='Аналитическое решение')
plt.plot(t_test, y_pred, '--', label='PINN решение')
plt.xlabel('Время')
plt.ylabel('Смещение')
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(loss_history)
plt.xlabel('Эпоха')
plt.ylabel('Лосс (лог. масштаб)')
plt.title('История обучения')
plt.tight_layout()
plt.show()